### **ПРАКТИЧЕСКОЕ ЗАДАНИЕ**

**Задание**

Задание

1. Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

2. (дополнительно необязательная задача)на выбор взять

https://huggingface.co/datasets/sberquad

https://huggingface.co/datasets/blinoff/medical_qa_ru_data

натренировать любую модель для вопросно ответной системы

как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

In [ ]:
# Импортируем библиотеки

!pip install datasets
!pip install -q transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizerFast
from datasets import load_dataset
import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')
import torch
import tensorflow as tf
from tqdm import tqdm
from datasets import load_metric
from datasets import ClassLabel, Value
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import DataLoader
!pip install evaluate
import evaluate

In [2]:
# Загрузим датасет

train, test = load_dataset('merionum/ru_paraphraser', split=['train', 'test'])

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train

Dataset({
    features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
    num_rows: 7227
})

In [4]:
# Сформируем датафрейм

df_train = pd.DataFrame(train, columns=['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'])
df_test = pd.DataFrame(test, columns=['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'])

# 1: precise paraphrases
# 0: near paraphrases
# -1: non-paraphrases

In [5]:
df_train.shape, df_test.shape

((7227, 6), (1924, 6))

In [6]:
# Посмотрим на распределение классов на трейне

df_train['class'].value_counts()

0     2957
-1    2582
1     1688
Name: class, dtype: int64

In [7]:
# Посмотрим на распределение классов на тесте

df_test['class'].value_counts()

0     778
-1    772
1     374
Name: class, dtype: int64

In [8]:
df_train.head(10)

,id,id_1,id_2,text_1,text_2,class
0,1,201,8159,Полицейским разрешат стрелять на поражение по гражданам с травматикой.,Полиции могут разрешить стрелять по хулиганам с травматикой.,0
1,2,202,8158,Право полицейских на проникновение в жилище решили ограничить.,Правила внесудебного проникновения полицейских в жилище уточнят.,0
2,3,273,8167,Президент Египта ввел чрезвычайное положение в мятежных городах.,Власти Египта угрожают ввести в стране чрезвычайное положение.,0
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос трудоустройства на родине.,Самолеты МЧС вывезут россиян из разрушенной Сирии.,-1
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с россиянами на борту.,Самолеты МЧС вывезут россиян из разрушенной Сирии.,0
5,6,154,8153,Приставы соберут отпечатки пальцев российских должников.,Приставы снимут отпечатки пальцев у злостных неплательщиков.,1
6,7,288,8170,На саратовского дебошира с борта самолета Москва - Хургада заведено дело.,Саратовский дебошир отказывается возвращаться домой из Египта.,-1
7,8,237,8163,ЦИК хочет отказаться от электронной системы подсчета голосов.,ЦИК может отказаться от электронных средств подсчета голосов.,0
8,9,192,8156,Суд Петербурга оставил на потом дело о гибели подростка в полиции.,"Лондонский Гайд-парк - это не место для митингов, а прежде всего парк.",-1
9,10,127,8149,Страны ОПЕК сократили добычу нефти на 1 млн баррелей в день.,Обама продлил полномочия НАСА по сотрудничеству с Россией.,-1


In [9]:
# Подключим GPU

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
# Загрузим модель, которая натренирована распознавать эмоциональную окраску на 3 класса
# Посмотрим как она справится с задачей парафраз
# 0: NEUTRAL
# 1: POSITIVE
# 2: NEGATIVE

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', num_labels=3).to(device)

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [11]:
# Посмотрим на параметры модели

print(model)
print(f'Total parameters: {model.num_parameters()}')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [14]:
# Токенизируем предложения

def tokenize_function(data):
    return tokenizer(data['text_1'], data['text_2'], padding='max_length', truncation=True)

tokenized_datasets_train = train.map(tokenize_function, batched=True)
tokenized_datasets_test = test.map(tokenize_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/2 [00:00<?, ?ba/s]

In [15]:
tokenized_datasets_train

Dataset({
    features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7227
})

In [16]:
# Удалим ненужные столбцы и преобразуем метки классов

labels_train = tokenized_datasets_train['class']
labels_test = tokenized_datasets_test['class']

labels_train = [2 if (lbl == '-1') else int(lbl) for lbl in labels_train]
labels_test = [2 if (lbl == '-1') else int(lbl) for lbl in labels_test]

tokenized_datasets_train = tokenized_datasets_train.remove_columns(['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'])
tokenized_datasets_test = tokenized_datasets_test.remove_columns(['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'])

tokenized_datasets_train = tokenized_datasets_train.add_column('labels', labels_train)
tokenized_datasets_test = tokenized_datasets_test.add_column('labels', labels_test)

tokenized_datasets_train.set_format('torch')
tokenized_datasets_test.set_format('torch')

In [17]:
# Создадим даталоудер

train_dataloader = DataLoader(tokenized_datasets_train, batch_size=1)
eval_dataloader = DataLoader(tokenized_datasets_test, batch_size=1)

In [18]:
# Посмотрим на один экземпляр из даталоудера

next(iter(train_dataloader))

{'input_ids': tensor([[  101, 34784,  2390, 62951, 14718,   801, 16661,   797, 18493,   336,
          45843,   915,   132,   102, 94695,  1677, 30146, 14718,   797, 99271,
            347,   336, 45843,   915,   132,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1]]),
 'labels': tensor([0])}

In [19]:
# Зададим параметры обучения

num_epochs = 3
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name='linear', 
                             optimizer=optimizer, 
                             num_warmup_steps=0, 
                             num_training_steps=num_training_steps)

In [20]:
# Проведем обучение модели

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [21]:
# Оценим метрику качества

metric = evaluate.load('accuracy')
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.40332640332640335}

**ВЫВОД:**

Качество классификации парафраз даже после дообучения оказалось довольно плохим.

Попробуем обучить модель на 2-х классах.

In [22]:
# Выберем другую модель для обучения

model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2, ignore_mismatched_sizes=True, problem_type = 'single_label_classification').to(device)
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [23]:
# Токенизируем предложения

def tokenize_function(data):
    return tokenizer(data['text_1'], data['text_2'], padding='max_length', truncation=True)

tokenized_datasets_train = train.map(tokenize_function, batched=True)
tokenized_datasets_test = test.map(tokenize_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [24]:
# Удалим ненужные столбцы и преобразуем метки классов

labels_train = tokenized_datasets_train['class']
labels_test = tokenized_datasets_test['class']

labels_train = [1 if (lbl == '0') else int(lbl) for lbl in labels_train]
labels_test = [1 if (lbl == '0') else int(lbl) for lbl in labels_test]

labels_train = [0 if (lbl == -1) else lbl for lbl in labels_train]
labels_test = [0 if (lbl == -1) else lbl for lbl in labels_test]

tokenized_datasets_train = tokenized_datasets_train.remove_columns(['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'])
tokenized_datasets_test = tokenized_datasets_test.remove_columns(['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'])

tokenized_datasets_train = tokenized_datasets_train.add_column('labels', labels_train)
tokenized_datasets_test = tokenized_datasets_test.add_column('labels', labels_test)

tokenized_datasets_train.set_format('torch')
tokenized_datasets_test.set_format('torch')

In [25]:
tokenized_datasets_train['labels']

tensor([1, 1, 1,  ..., 0, 0, 0])

In [26]:
# Создадим даталоудер

train_dataloader = DataLoader(tokenized_datasets_train, batch_size=1)
eval_dataloader = DataLoader(tokenized_datasets_test, batch_size=1)

In [27]:
# Зададим параметры обучения

num_epochs = 3
optimizer = AdamW(model.parameters(), lr=3e-5)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name='linear', 
                             optimizer=optimizer, 
                             num_warmup_steps=0, 
                             num_training_steps=num_training_steps)

In [31]:
print(model)
print(f'Total parameters: {model.num_parameters()}')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [29]:
# Проведем обучение модели

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 21681/21681 [1:00:16<00:00,  5.99it/s]

In [30]:
# Оценим метрику качества

metric = evaluate.load('accuracy')
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()

{'accuracy': 0.5987525987525988}

In [32]:
# Функция предсказания класса парафразы

def check_paraphrase(seq_1, seq_2, classes):
  tokens = tokenizer(seq_1, seq_2, return_tensors='pt').to(device)
  tokens_classification_logits = model(**tokens)[0]
  results = tf.nn.softmax(tokens_classification_logits.detach().cpu().numpy(), axis=1).numpy()[0]

  print(f'\n{seq_1} \n{seq_2}')
  for i in range(len(classes)):
      print(f'Class {classes[i]}: {int(round(results[i] * 100))}%')

In [34]:
# Зададим параметры и посмотрим на результат предсказания

classes = [0, 1]

sequence_0 = df_train.text_1[5]
sequence_1 = df_train.text_2[5]
sequence_2 = df_train.text_1[1]
sequence_3 = df_train.text_2[1]
sequence_4 = df_train.text_1[8]
sequence_5 = df_train.text_2[8]

check_paraphrase(sequence_0, sequence_1, classes)
print('True class: 1')
check_paraphrase(sequence_2, sequence_3, classes)
print('True class: 1')
check_paraphrase(sequence_4, sequence_5, classes)
print('True class: 0')


Приставы соберут отпечатки пальцев российских должников. 
Приставы снимут отпечатки пальцев у злостных неплательщиков.
Class 0: 39%
Class 1: 61%
True class: 1

Право полицейских на проникновение в жилище решили ограничить. 
Правила внесудебного проникновения полицейских в жилище уточнят.
Class 0: 39%
Class 1: 61%
True class: 1

Суд Петербурга оставил на потом дело о гибели подростка в полиции. 
Лондонский Гайд-парк - это не место для митингов, а прежде всего парк.
Class 0: 39%
Class 1: 61%
True class: 0


**ВЫВОД:**

Метрика качества стала выше, но оставляет желать лучшего.

Для дальнейшего ее повышения стоит сделать подбор гиперпараментров.